In [2]:
!pip install pandas

import pandas as pd
import numpy as np
import requests # library to handle requests
import json # library to handle JSON files
from pandas.io.json import json_normalize
from datetime import datetime, date, timedelta
import os.path 
from os import path

In [3]:
#returns weather json data from weatherstack api
def request_weather_json(key, date, location):
    params = {
        'access_key': key,
        'query': location,
        'historical_date': date,
        'hourly': '1',
        'interval': '1'
        }

    api_result = requests.get('https://api.weatherstack.com/historical', params)
    api_response = api_result.json()
    print(api_response['location'])
    return api_response

In [4]:
#Formats weather json data and outputs to detailed_weather_report.csv
def append_weather_data(json):
    json_historical = json_normalize(data=json['historical'][date], errors='ignore')
    detailed_weather_report = json_normalize(data=json_historical['hourly'][0], meta=['day'], meta_prefix='pre', errors='ignore')
    detailed_weather_report['location'] = json['location']['name']
    detailed_weather_report['date'] = date
    
    if path.exists("detailed_weather_report.csv"):
        detailed_weather_report.to_csv('detailed_weather_report.csv', mode='a', header=False)
    else:
        detailed_weather_report.to_csv('detailed_weather_report.csv')

In [24]:
def create_report():
    detailed_weather_df = pd.read_csv("detailed_weather_report.csv")
    weather_df = detailed_weather_df[detailed_weather_df['Unnamed: 0'].between(9,17)]
    weather_df['precip'].replace({0:np.nan}, inplace=True)
    weather_df = weather_df.groupby(['location','date']).agg({'precip':'count', 'temperature': 'max'}).reset_index().rename(columns={'precip':'RH', 'temperature':'TMP'})
    weather_df['RH%'] = round(weather_df['RH']/9*100,0)
    weather_df = weather_df.merge(regions, left_on='location', right_on='Region' )
    weather_summary_df = weather_df
    weather_df = weather_df.pivot_table(index=['RegionID','Region'], columns='date', values=['RH%', 'TMP']).stack(0).reset_index()
    weather_df.rename(columns={"level_2": "Type"}, inplace=True)
    weather_df.to_csv('temperature_and_rainfall_by_region.csv')
    
    weather_summary_df = weather_summary_df.groupby(['date']).agg({'RH%':'mean', 'TMP': 'mean'}).T
    weather_summary_df.to_csv('temperature_and_rainfall.csv')
    weather_df

In [6]:
today = datetime.now().date()
yesterday = today - timedelta(days=-1)
week_start = today - timedelta(days=today.weekday()+1)

# this will give you a list containing all of the dates
week_days = [(week_start + timedelta(days=x)).isoformat() for x in range((today-week_start).days)]

week_days

['2020-01-19', '2020-01-20']

In [25]:
key = '8c5dad538cf97d8347dbfda0e9369f25'
#dates = ['2020-01-01', '2020-01-02']
dates = week_days
#regions = ['Glasgow, UK', 'leeming, UK', 'London, UK', 'Belfast']
regions_dict = {'RegionID':[1, 3, 6, 8],
                'Region_search':['Glasgow, UK', 'Leeming, UK', 'London, UK', 'Belfast'],
                'Region':['Glasgow', 'Leeming', 'London', 'Belfast']}
regions = pd.DataFrame(regions_dict)

# Deletes existing file
os.remove('detailed_weather_report.csv')

for region in regions_dict['Region_search']:
    for date in dates:    
        json = request_weather_json(key, date, region)
        append_weather_data(json)
        
create_report()

{'name': 'Glasgow', 'country': 'United Kingdom', 'region': 'Glasgow City', 'lat': '55.862', 'lon': '-4.247', 'timezone_id': 'Europe/London', 'localtime': '2020-01-21 14:45', 'localtime_epoch': 1579617900, 'utc_offset': '0.0'}
{'name': 'Glasgow', 'country': 'United Kingdom', 'region': 'Glasgow City', 'lat': '55.862', 'lon': '-4.247', 'timezone_id': 'Europe/London', 'localtime': '2020-01-21 14:45', 'localtime_epoch': 1579617900, 'utc_offset': '0.0'}
{'name': 'Leeming', 'country': 'United Kingdom', 'region': 'West Yorkshire', 'lat': '53.807', 'lon': '-1.933', 'timezone_id': 'Europe/London', 'localtime': '2020-01-21 14:45', 'localtime_epoch': 1579617900, 'utc_offset': '0.0'}
{'name': 'Leeming', 'country': 'United Kingdom', 'region': 'West Yorkshire', 'lat': '53.807', 'lon': '-1.933', 'timezone_id': 'Europe/London', 'localtime': '2020-01-21 14:45', 'localtime_epoch': 1579617900, 'utc_offset': '0.0'}
{'name': 'London', 'country': 'United Kingdom', 'region': 'City of London, Greater London', 